# 01 - Geração de Perfis Cognitivos para Simulação SINKT

Este notebook implementa a **Etapa 1** do pipeline SINKT: criação de perfis cognitivos fundamentados em modelos clássicos de Knowledge Tracing (BKT - Bayesian Knowledge Tracing).

## Objetivo
Criar 6+ perfis cognitivos distintos que representam diferentes tipos de estudantes, cada um com características únicas de aprendizagem.

## Saída
- `data/output/profiles.json`: Arquivo JSON contendo todos os perfis cognitivos
- `data/output/profiles_analysis.md`: Documentação e análise dos perfis gerados

## Importação de Bibliotecas

In [ ]:
import json
import os
from datetime import datetime
from typing import Dict, List, Any
import numpy as np
import pandas as pd

print("✅ Bibliotecas importadas com sucesso")

## Definição dos Perfis Cognitivos Enriquecidos

Cada perfil é caracterizado por 9 parâmetros fundamentais baseados em modelos cognitivos reais:

### Parâmetros BKT (Knowledge Tracing)
- **mastery_init_level**: Nível inicial de domínio (0-1) - conhecimento prévio
- **learn_rate**: Taxa de aprendizagem (0-0.1) - velocidade de aquisição de conhecimento
- **slip**: Probabilidade de erro quando sabe (0-0.2) - erros por distração/pressa
- **guess**: Probabilidade de acerto quando não sabe (0-0.3) - acertos por sorte

### Parâmetros Cognitivos
- **logic_skill**: Habilidade lógica (0-1) - raciocínio formal e algorítmico
- **reading_skill**: Habilidade de leitura (0-1) - compreensão de texto
- **memory_capacity**: Capacidade de memória (0-1) - retenção de informações
- **tech_familiarity**: Familiaridade com tecnologia (0-1) - experiência prévia
- **learning_consistency**: Consistência de aprendizado (0-1) - regularidade e disciplina

In [ ]:
profiles_data = [
    {
        "id": "balanced",
        "nome": "Estudante Equilibrado",
        "descricao": "Perfil médio com habilidades balanceadas. Representa o estudante típico com desenvolvimento uniforme.",
        "mastery_init_level": 0.50,
        "learn_rate": 0.035,
        "slip": 0.12,
        "guess": 0.15,
        "logic_skill": 0.55,
        "reading_skill": 0.55,
        "memory_capacity": 0.55,
        "tech_familiarity": 0.50,
        "learning_consistency": 0.60
    },
    {
        "id": "quick_learner",
        "nome": "Aprendiz Rápido",
        "descricao": "Alta capacidade de aprendizagem e domínio inicial. Adquire conceitos rapidamente com poucos erros.",
        "mastery_init_level": 0.70,
        "learn_rate": 0.075,
        "slip": 0.08,
        "guess": 0.10,
        "logic_skill": 0.75,
        "reading_skill": 0.70,
        "memory_capacity": 0.80,
        "tech_familiarity": 0.75,
        "learning_consistency": 0.85
    },
    {
        "id": "careful",
        "nome": "Estudante Cuidadoso",
        "descricao": "Baixa taxa de erros quando sabe, mas aprende de forma gradual. Meticuloso e detalhista.",
        "mastery_init_level": 0.45,
        "learn_rate": 0.020,
        "slip": 0.06,
        "guess": 0.08,
        "logic_skill": 0.65,
        "reading_skill": 0.75,
        "memory_capacity": 0.70,
        "tech_familiarity": 0.45,
        "learning_consistency": 0.90
    },
    {
        "id": "struggling",
        "nome": "Estudante com Dificuldades",
        "descricao": "Baixo domínio inicial e lenta aprendizagem. Necessita mais apoio e tempo para absorver conceitos.",
        "mastery_init_level": 0.25,
        "learn_rate": 0.012,
        "slip": 0.20,
        "guess": 0.22,
        "logic_skill": 0.30,
        "reading_skill": 0.35,
        "memory_capacity": 0.40,
        "tech_familiarity": 0.20,
        "learning_consistency": 0.40
    },
    {
        "id": "logical",
        "nome": "Pensador Lógico",
        "descricao": "Excelente habilidade lógica mas com dificuldades em leitura. Excelente em problemas algorítmicos.",
        "mastery_init_level": 0.55,
        "learn_rate": 0.045,
        "slip": 0.10,
        "guess": 0.08,
        "logic_skill": 0.90,
        "reading_skill": 0.25,
        "memory_capacity": 0.60,
        "tech_familiarity": 0.65,
        "learning_consistency": 0.75
    },
    {
        "id": "intuitive",
        "nome": "Estudante Intuitivo",
        "descricao": "Boa intuição e habilidade de leitura, mas dificuldade com formalismo lógico. Aprende por exemplo.",
        "mastery_init_level": 0.40,
        "learn_rate": 0.040,
        "slip": 0.14,
        "guess": 0.18,
        "logic_skill": 0.35,
        "reading_skill": 0.85,
        "memory_capacity": 0.65,
        "tech_familiarity": 0.40,
        "learning_consistency": 0.65
    },
    {
        "id": "perfectionist",
        "nome": "Perfeccionista",
        "descricao": "Muito rigoroso e exigente consigo mesmo. Aprende profundamente mas pode ser lento.",
        "mastery_init_level": 0.60,
        "learn_rate": 0.025,
        "slip": 0.05,
        "guess": 0.05,
        "logic_skill": 0.80,
        "reading_skill": 0.80,
        "memory_capacity": 0.85,
        "tech_familiarity": 0.60,
        "learning_consistency": 0.95
    }
]

print(f"✅ {len(profiles_data)} perfis cognitivos definidos")

## Validação dos Perfis

In [ ]:
def validate_profiles(profiles: List[Dict[str, Any]]) -> List[str]:
    """Valida os perfis cognitivos."""
    errors = []
    
    required_fields = ['id', 'nome', 'descricao', 'mastery_init_level', 'learn_rate', 
                       'slip', 'guess', 'logic_skill', 'reading_skill', 'memory_capacity',
                       'tech_familiarity', 'learning_consistency']
    
    numeric_fields = ['mastery_init_level', 'learn_rate', 'slip', 'guess', 
                      'logic_skill', 'reading_skill', 'memory_capacity',
                      'tech_familiarity', 'learning_consistency']
    
    profile_ids = set()
    
    for i, profile in enumerate(profiles):
        # Verifica campos obrigatórios
        for field in required_fields:
            if field not in profile:
                errors.append(f"Perfil {i}: Campo '{field}' ausente")
        
        # Verifica ID único
        if 'id' in profile:
            if profile['id'] in profile_ids:
                errors.append(f"Perfil {i}: ID '{profile['id']}' duplicado")
            profile_ids.add(profile['id'])
        
        # Verifica ranges dos parâmetros numéricos
        for field in numeric_fields:
            if field in profile:
                value = profile[field]
                if not isinstance(value, (int, float)):
                    errors.append(f"Perfil {profile.get('id', i)}: '{field}' deve ser numérico")
                elif value < 0 or value > 1:
                    errors.append(f"Perfil {profile.get('id', i)}: '{field}' = {value} fora do range [0, 1]")
    
    return errors

# Executar validação
validation_errors = validate_profiles(profiles_data)

if validation_errors:
    print("❌ Erros de validação encontrados:")
    for error in validation_errors:
        print(f"  - {error}")
else:
    print("✅ Todos os perfis são válidos!")

## Análise de Coerência entre Parâmetros

In [ ]:
def analyze_profile_coherence(profiles: List[Dict[str, Any]]) -> pd.DataFrame:
    """Analisa coerência entre parâmetros dos perfis."""
    data = []
    
    for profile in profiles:
        # Verificar coerência: aprendizes rápidos devem ter slip baixo
        coherence_score = 1.0
        issues = []
        
        # Regra 1: learn_rate alto → slip baixo
        if profile['learn_rate'] > 0.05 and profile['slip'] > 0.15:
            coherence_score -= 0.2
            issues.append("learn_rate alto com slip alto")
        
        # Regra 2: logic_skill alto → guess baixo
        if profile['logic_skill'] > 0.7 and profile['guess'] > 0.15:
            coherence_score -= 0.15
            issues.append("logic_skill alto com guess alto")
        
        # Regra 3: memory_capacity alto → learning_consistency alto
        if profile['memory_capacity'] > 0.7 and profile['learning_consistency'] < 0.6:
            coherence_score -= 0.15
            issues.append("memory_capacity alto com consistency baixa")
        
        data.append({
            'Perfil': profile['nome'],
            'ID': profile['id'],
            'Coerência': max(0, coherence_score),
            'Problemas': '; '.join(issues) if issues else 'Nenhum'
        })
    
    return pd.DataFrame(data)

coherence_df = analyze_profile_coherence(profiles_data)
print("\n📊 Análise de Coerência dos Perfis:\n")
print(coherence_df.to_string(index=False))

## Cálculo de Estatísticas dos Perfis

In [ ]:
def calculate_profile_statistics(profiles: List[Dict[str, Any]]) -> Dict[str, Dict[str, float]]:
    """Calcula estatísticas dos parâmetros dos perfis."""
    numeric_fields = ['mastery_init_level', 'learn_rate', 'slip', 'guess', 
                      'logic_skill', 'reading_skill', 'memory_capacity',
                      'tech_familiarity', 'learning_consistency']
    
    stats = {}
    
    for field in numeric_fields:
        values = [p[field] for p in profiles if field in p]
        if values:
            stats[field] = {
                'mean': float(np.mean(values)),
                'std': float(np.std(values)),
                'min': float(np.min(values)),
                'max': float(np.max(values))
            }
    
    return stats

# Calcular e exibir estatísticas
stats = calculate_profile_statistics(profiles_data)

print("\n📊 Estatísticas dos Parâmetros:\n")
for param, values in stats.items():
    print(f"  {param}:")
    print(f"    Média: {values['mean']:.3f} | Desvio: {values['std']:.3f} | Range: [{values['min']:.3f}, {values['max']:.3f}]")

## Criação do Arquivo JSON de Saída

In [ ]:
# Criar estrutura completa com metadados
output_data = {
    "metadata": {
        "description": "Conjunto de perfis cognitivos para simulação de estudantes SINKT",
        "version": "2.0.0",
        "created_at": datetime.now().isoformat(),
        "num_profiles": len(profiles_data),
        "parameters_count": 9,
        "model": "BKT (Bayesian Knowledge Tracing) + Cognitive Factors"
    },
    "profiles": {profile['id']: profile for profile in profiles_data},
    "statistics": stats
}

# Criar diretório de saída se não existir
os.makedirs('data/output', exist_ok=True)

# Salvar arquivo JSON
output_file = 'data/output/profiles.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"✅ Perfis salvos em: {output_file}")
print(f"📦 Total de perfis: {len(profiles_data)}")
print(f"📊 Total de parâmetros por perfil: 9")

## Resumo da Execução

In [ ]:
print("\n" + "="*70)
print("🎉 GERAÇÃO DE PERFIS COGNITIVOS CONCLUÍDA COM SUCESSO!")
print("="*70)
print(f"\n📁 Arquivo gerado:")
print(f"  - {output_file}")
print(f"\n📊 Perfis criados ({len(profiles_data)}):")
for profile in profiles_data:
    print(f"  ✓ {profile['nome']} ({profile['id']})")
print(f"\n✅ Próximo passo: Execute o notebook '02_geracao_estudantes.ipynb'")
print("="*70)